In [2]:
import numpy as np
import pandas as pd

In [3]:
def unpack(D, Dp, Ty="int32_t", dterm="logeta", n=8):
    if Dp%2 == 0 and n>1:
        return unpack(D, Dp//2, Ty, dterm, n//2)
    
    X = [[i]*D for i in range(8)]
    Y = [[-1]*8 for i in range(D)]
    Z= [-1]*(8*D)
    l = 0
    for i in range(8):
        for j in range(D):
            Z[l] = X[i][j]
            l += 1

    l = 0
    for i in range(D):
        for j in range(8):
            Y[i][j] = Z[l]
            l += 1

    ta = []
    tb = []
    for y in Y:
        y = pd.Series(y)
        c = dict(y.value_counts())
        ta.append(c)
        for key in c.keys():
            tb.append({key: c[key]})
            
    print("\nunsigned int i;\nfor(i=0;i<N/{};++i)\n{{\n".format(n))

    cp = 0
    cp_key = 0
    it = 0
    for y in Y:
        y = pd.Series(y)
        c = dict(y.value_counts())
        init = 0
        sorted_ = list(c.keys())
        sorted_.sort()
        for key in sorted_:
            if key >= n:
                break
            cp = cp%D
            if init == 0:
                print("    r->coeffs[{} * i + {}] {}= {}a[{} * i + {}]{};".format(n, key, "|" if cp else "", "(%s)"%(Ty) if cp else "", Dp, it, " << {}".format(cp) if cp else ""))
                init += c[key]
            else:            
                print("    r->coeffs[{} * i + {}] &= {};\n".format(n, cp_key, hex((2<<(D-1))-1)))
                print("    r->coeffs[{} * i + {}] = a[{} * i + {}]{};".format(n, key, Dp, it, " >> {}".format(init) if init else ""))
                init += c[key]


            if (cp_key == key):
                cp += c[key]
            else:
                cp = c[key]

            cp_key = key

        it += 1
    print("    r->coeffs[{} * i + {}] &= {};\n".format(n, cp_key, hex((2<<(D-1))-1)))

    
    for i in range(n):
        print("    r->coeffs[{2} * i + {0}] = (1 << ({1} - 1)) - r->coeffs[{2} * i + {0}];".format(i, dterm, n))
        
    print("\n}")

In [4]:
unpack(21, 21, "int32_t",  "COEFF_BIT_SIZE")


unsigned int i;
for(i=0;i<N/8;++i)
{

    r->coeffs[8 * i + 0] = a[21 * i + 0];
    r->coeffs[8 * i + 0] |= (int32_t)a[21 * i + 1] << 8;
    r->coeffs[8 * i + 0] |= (int32_t)a[21 * i + 2] << 16;
    r->coeffs[8 * i + 0] &= 0x1fffff;

    r->coeffs[8 * i + 1] = a[21 * i + 2] >> 5;
    r->coeffs[8 * i + 1] |= (int32_t)a[21 * i + 3] << 3;
    r->coeffs[8 * i + 1] |= (int32_t)a[21 * i + 4] << 11;
    r->coeffs[8 * i + 1] |= (int32_t)a[21 * i + 5] << 19;
    r->coeffs[8 * i + 1] &= 0x1fffff;

    r->coeffs[8 * i + 2] = a[21 * i + 5] >> 2;
    r->coeffs[8 * i + 2] |= (int32_t)a[21 * i + 6] << 6;
    r->coeffs[8 * i + 2] |= (int32_t)a[21 * i + 7] << 14;
    r->coeffs[8 * i + 2] &= 0x1fffff;

    r->coeffs[8 * i + 3] = a[21 * i + 7] >> 7;
    r->coeffs[8 * i + 3] |= (int32_t)a[21 * i + 8] << 1;
    r->coeffs[8 * i + 3] |= (int32_t)a[21 * i + 9] << 9;
    r->coeffs[8 * i + 3] |= (int32_t)a[21 * i + 10] << 17;
    r->coeffs[8 * i + 3] &= 0x1fffff;

    r->coeffs[8 * i + 4] = a[21 * i + 10] >> 4

In [21]:
unpack(3, 3, "Q_SIZE")


unsigned int i;
for(i=0;i<N/8;++i)
{

    r->coeffs[8 * i + 0] = a[3 * i + 0];
    r->coeffs[8 * i + 0] &= 0x7;

    r->coeffs[8 * i + 1] = a[3 * i + 0] >> 3;
    r->coeffs[8 * i + 1] &= 0x7;

    r->coeffs[8 * i + 2] = a[3 * i + 0] >> 6;
    r->coeffs[8 * i + 2] |= (Q_SIZE)a[3 * i + 1] << 2;
    r->coeffs[8 * i + 2] &= 0x7;

    r->coeffs[8 * i + 3] = a[3 * i + 1] >> 1;
    r->coeffs[8 * i + 3] &= 0x7;

    r->coeffs[8 * i + 4] = a[3 * i + 1] >> 4;
    r->coeffs[8 * i + 4] &= 0x7;

    r->coeffs[8 * i + 5] = a[3 * i + 1] >> 7;
    r->coeffs[8 * i + 5] |= (Q_SIZE)a[3 * i + 2] << 1;
    r->coeffs[8 * i + 5] &= 0x7;

    r->coeffs[8 * i + 6] = a[3 * i + 2] >> 2;
    r->coeffs[8 * i + 6] &= 0x7;

    r->coeffs[8 * i + 7] = a[3 * i + 2] >> 5;
    r->coeffs[8 * i + 7] &= 0x7;

    r->coeffs[8 * i + 0] = (1 << (logeta - 1)) - r->coeffs[8 * i + 0];
    r->coeffs[8 * i + 1] = (1 << (logeta - 1)) - r->coeffs[8 * i + 1];
    r->coeffs[8 * i + 2] = (1 << (logeta - 1)) - r->coeffs[8 * i + 2];
   

In [22]:
unpack(18, 18, "Q_SIZE")


unsigned int i;
for(i=0;i<N/4;++i)
{

    r->coeffs[4 * i + 0] = a[9 * i + 0];
    r->coeffs[4 * i + 0] |= (Q_SIZE)a[9 * i + 1] << 8;
    r->coeffs[4 * i + 0] |= (Q_SIZE)a[9 * i + 2] << 16;
    r->coeffs[4 * i + 0] &= 0x3ffff;

    r->coeffs[4 * i + 1] = a[9 * i + 2] >> 2;
    r->coeffs[4 * i + 1] |= (Q_SIZE)a[9 * i + 3] << 6;
    r->coeffs[4 * i + 1] |= (Q_SIZE)a[9 * i + 4] << 14;
    r->coeffs[4 * i + 1] &= 0x3ffff;

    r->coeffs[4 * i + 2] = a[9 * i + 4] >> 4;
    r->coeffs[4 * i + 2] |= (Q_SIZE)a[9 * i + 5] << 4;
    r->coeffs[4 * i + 2] |= (Q_SIZE)a[9 * i + 6] << 12;
    r->coeffs[4 * i + 2] &= 0x3ffff;

    r->coeffs[4 * i + 3] = a[9 * i + 6] >> 6;
    r->coeffs[4 * i + 3] |= (Q_SIZE)a[9 * i + 7] << 2;
    r->coeffs[4 * i + 3] |= (Q_SIZE)a[9 * i + 8] << 10;
    r->coeffs[4 * i + 3] &= 0x3ffff;

    r->coeffs[4 * i + 0] = (1 << (logeta - 1)) - r->coeffs[4 * i + 0];
    r->coeffs[4 * i + 1] = (1 << (logeta - 1)) - r->coeffs[4 * i + 1];
    r->coeffs[4 * i + 2] = (1 << (logeta - 1

In [23]:
unpack(21, 21, "Q_SIZE")


unsigned int i;
for(i=0;i<N/8;++i)
{

    r->coeffs[8 * i + 0] = a[21 * i + 0];
    r->coeffs[8 * i + 0] |= (Q_SIZE)a[21 * i + 1] << 8;
    r->coeffs[8 * i + 0] |= (Q_SIZE)a[21 * i + 2] << 16;
    r->coeffs[8 * i + 0] &= 0x1fffff;

    r->coeffs[8 * i + 1] = a[21 * i + 2] >> 5;
    r->coeffs[8 * i + 1] |= (Q_SIZE)a[21 * i + 3] << 3;
    r->coeffs[8 * i + 1] |= (Q_SIZE)a[21 * i + 4] << 11;
    r->coeffs[8 * i + 1] |= (Q_SIZE)a[21 * i + 5] << 19;
    r->coeffs[8 * i + 1] &= 0x1fffff;

    r->coeffs[8 * i + 2] = a[21 * i + 5] >> 2;
    r->coeffs[8 * i + 2] |= (Q_SIZE)a[21 * i + 6] << 6;
    r->coeffs[8 * i + 2] |= (Q_SIZE)a[21 * i + 7] << 14;
    r->coeffs[8 * i + 2] &= 0x1fffff;

    r->coeffs[8 * i + 3] = a[21 * i + 7] >> 7;
    r->coeffs[8 * i + 3] |= (Q_SIZE)a[21 * i + 8] << 1;
    r->coeffs[8 * i + 3] |= (Q_SIZE)a[21 * i + 9] << 9;
    r->coeffs[8 * i + 3] |= (Q_SIZE)a[21 * i + 10] << 17;
    r->coeffs[8 * i + 3] &= 0x1fffff;

    r->coeffs[8 * i + 4] = a[21 * i + 10] >> 4;
    r->c

In [24]:
unpack(22, 22, "Q_SIZE")


unsigned int i;
for(i=0;i<N/4;++i)
{

    r->coeffs[4 * i + 0] = a[11 * i + 0];
    r->coeffs[4 * i + 0] |= (Q_SIZE)a[11 * i + 1] << 8;
    r->coeffs[4 * i + 0] |= (Q_SIZE)a[11 * i + 2] << 16;
    r->coeffs[4 * i + 0] &= 0x3fffff;

    r->coeffs[4 * i + 1] = a[11 * i + 2] >> 6;
    r->coeffs[4 * i + 1] |= (Q_SIZE)a[11 * i + 3] << 2;
    r->coeffs[4 * i + 1] |= (Q_SIZE)a[11 * i + 4] << 10;
    r->coeffs[4 * i + 1] |= (Q_SIZE)a[11 * i + 5] << 18;
    r->coeffs[4 * i + 1] &= 0x3fffff;

    r->coeffs[4 * i + 2] = a[11 * i + 5] >> 4;
    r->coeffs[4 * i + 2] |= (Q_SIZE)a[11 * i + 6] << 4;
    r->coeffs[4 * i + 2] |= (Q_SIZE)a[11 * i + 7] << 12;
    r->coeffs[4 * i + 2] |= (Q_SIZE)a[11 * i + 8] << 20;
    r->coeffs[4 * i + 2] &= 0x3fffff;

    r->coeffs[4 * i + 3] = a[11 * i + 8] >> 2;
    r->coeffs[4 * i + 3] |= (Q_SIZE)a[11 * i + 9] << 6;
    r->coeffs[4 * i + 3] |= (Q_SIZE)a[11 * i + 10] << 14;
    r->coeffs[4 * i + 3] &= 0x3fffff;

    r->coeffs[4 * i + 0] = (1 << (logeta - 1)) - r->coe

In [25]:
unpack(23, 23, "Q_SIZE")


unsigned int i;
for(i=0;i<N/8;++i)
{

    r->coeffs[8 * i + 0] = a[23 * i + 0];
    r->coeffs[8 * i + 0] |= (Q_SIZE)a[23 * i + 1] << 8;
    r->coeffs[8 * i + 0] |= (Q_SIZE)a[23 * i + 2] << 16;
    r->coeffs[8 * i + 0] &= 0x7fffff;

    r->coeffs[8 * i + 1] = a[23 * i + 2] >> 7;
    r->coeffs[8 * i + 1] |= (Q_SIZE)a[23 * i + 3] << 1;
    r->coeffs[8 * i + 1] |= (Q_SIZE)a[23 * i + 4] << 9;
    r->coeffs[8 * i + 1] |= (Q_SIZE)a[23 * i + 5] << 17;
    r->coeffs[8 * i + 1] &= 0x7fffff;

    r->coeffs[8 * i + 2] = a[23 * i + 5] >> 6;
    r->coeffs[8 * i + 2] |= (Q_SIZE)a[23 * i + 6] << 2;
    r->coeffs[8 * i + 2] |= (Q_SIZE)a[23 * i + 7] << 10;
    r->coeffs[8 * i + 2] |= (Q_SIZE)a[23 * i + 8] << 18;
    r->coeffs[8 * i + 2] &= 0x7fffff;

    r->coeffs[8 * i + 3] = a[23 * i + 8] >> 5;
    r->coeffs[8 * i + 3] |= (Q_SIZE)a[23 * i + 9] << 3;
    r->coeffs[8 * i + 3] |= (Q_SIZE)a[23 * i + 10] << 11;
    r->coeffs[8 * i + 3] |= (Q_SIZE)a[23 * i + 11] << 19;
    r->coeffs[8 * i + 3] &= 0x7fffff;

In [26]:
unpack(24, 24, "Q_SIZE")


unsigned int i;
for(i=0;i<N/1;++i)
{

    r->coeffs[1 * i + 0] = a[3 * i + 0];
    r->coeffs[1 * i + 0] |= (Q_SIZE)a[3 * i + 1] << 8;
    r->coeffs[1 * i + 0] |= (Q_SIZE)a[3 * i + 2] << 16;
    r->coeffs[1 * i + 0] &= 0xffffff;

    r->coeffs[1 * i + 0] = (1 << (logeta - 1)) - r->coeffs[1 * i + 0];

}


In [27]:
unpack(25, 25, "Q_SIZE")


unsigned int i;
for(i=0;i<N/8;++i)
{

    r->coeffs[8 * i + 0] = a[25 * i + 0];
    r->coeffs[8 * i + 0] |= (Q_SIZE)a[25 * i + 1] << 8;
    r->coeffs[8 * i + 0] |= (Q_SIZE)a[25 * i + 2] << 16;
    r->coeffs[8 * i + 0] |= (Q_SIZE)a[25 * i + 3] << 24;
    r->coeffs[8 * i + 0] &= 0x1ffffff;

    r->coeffs[8 * i + 1] = a[25 * i + 3] >> 1;
    r->coeffs[8 * i + 1] |= (Q_SIZE)a[25 * i + 4] << 7;
    r->coeffs[8 * i + 1] |= (Q_SIZE)a[25 * i + 5] << 15;
    r->coeffs[8 * i + 1] |= (Q_SIZE)a[25 * i + 6] << 23;
    r->coeffs[8 * i + 1] &= 0x1ffffff;

    r->coeffs[8 * i + 2] = a[25 * i + 6] >> 2;
    r->coeffs[8 * i + 2] |= (Q_SIZE)a[25 * i + 7] << 6;
    r->coeffs[8 * i + 2] |= (Q_SIZE)a[25 * i + 8] << 14;
    r->coeffs[8 * i + 2] |= (Q_SIZE)a[25 * i + 9] << 22;
    r->coeffs[8 * i + 2] &= 0x1ffffff;

    r->coeffs[8 * i + 3] = a[25 * i + 9] >> 3;
    r->coeffs[8 * i + 3] |= (Q_SIZE)a[25 * i + 10] << 5;
    r->coeffs[8 * i + 3] |= (Q_SIZE)a[25 * i + 11] << 13;
    r->coeffs[8 * i + 3] |= (Q_SI